# Tonto el que lo lea

In [ ]:
El conjunto de datos está constituido por palabras del inglés y del catalán. Es el mismo conjunto de palabras para los dos idiomas, cada palabra en un idioma tiene su traducción en el otro.

In [3]:
import pandas
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

import numpy as np
from sklearn.svm import SVC

Primero cargamos el conjunto de datos en el programa usando la libreria pandas. Debido a que ya sabemos que hay una columna de indices que no aporta información relevante, la eliminamos seguidamente.

In [12]:
df = pandas.read_csv("data/data.csv", delim_whitespace = True)
df = df.drop(['0'], axis = 1)
print(df)

         catala     angles
0           com         as
1          seva        his
2           que       that
3           ell         he
4           era        was
..          ...        ...
983         nas       nose
984      plural     plural
985      còlera      anger
986  reclamació      claim
987   continent  continent

[988 rows x 2 columns]


El conjunto de datos está constituido por dos columnas, una de palabras en inglés i otra de su traducción en catalan. Si queremos entrenar un svm para que sea capaz de distinguir si una palabra es inglesa o catalana necesitamos formatear los datos para que en una columna tengamos todos los datos crudos, es decir las palabras, y su resolución, es decir su procedencia.

In [ ]:
#Funcion que crea las columnas de datos y de resoluciones
def reformat(dataFrame):
    dataFrame['y']=dataFrame.columns[0]
    dataFrame.rename(columns={dataFrame.columns[0]: 'word'}, inplace=True)
    return dataFrame;

#Separamos las palabras por idioma
catala, angles= df.filter(['catala'], axis=1), df.filter(['angles'], axis=1)
#Reformateamos
catala=reformat(catala)
angles=reformat(angles)
#Concatenamos los dos conjuntos de datos
wordsDF=pd.concat([catala,angles], axis=0)
#Mezclamos las filas para que las palabras de cada idioma no se encuentren todas juntas.
wordsDF = wordsDF.sample(frac=1).reset_index(drop=True)
wordsDF.to_csv('data/definitiveData.csv', index=False)
print(wordsDF.to_string)

El siguiente paso es determinar los parámetros sobre los cuales el svm se va a basar para hacer la distinción. Entonces, hay que analizar el idioma y encontrar sus características que lo diferencien.

longitud de palabra
ratio consonante/vocal
formacion th
formacion vocales iguales(oo, ee)

In [ ]:
#longitud de palabras
def contarLletres(lista):
    cantidadLetras = []
    for word in lista:
        cantidadLetras.append(len(word))
    return cantidadLetras
